In [1]:
from dotenv import load_dotenv
import os
import geopandas as gpd

load_dotenv()

True

In [2]:
from minio import Minio

minio_client = Minio(
	endpoint=os.getenv('S3_ENDPOINT'),
	access_key=os.getenv('ACCESS_KEY_ID'),
	secret_key=os.getenv('SECRET_ACCESS_KEY'),
	secure=True,
)

In [34]:
import geopandas as gpd

minio_client.fget_object(os.environ['BUCKET'], f'682f2d186a29eac175867330/catalog.v1.parquet', 'kk/catalog.parquet')

# gdf = gpd.read_parquet('/fastdata/kk/dataset/catalog.v1.parquet')
gdf = gpd.read_parquet('kk/catalog.parquet')

gdf

,type,stac_version,stac_extensions,datetime,id,bbox,geometry,assets,links,repository
0,Feature,1.0.0,[],2025-05-22 15:55:09.854311,README.md,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': 'd8623c18250c2458769849...,[],eotdl
1,Feature,1.0.0,[],2025-05-22 15:55:09.854900,eurocrops-all.pmtiles,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': '4d8d2b4b3c9783a16794fa...,[],eotdl
2,Feature,1.0.0,[],2025-05-22 15:55:12.118027,eurocrops-harmonized-only.fgb,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': 'ec25c7095771deb1ac0422...,[],eotdl
3,Feature,1.0.0,[],2025-05-22 15:55:26.061822,eurocrops-full.duckdb,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': '5bb84068d5c0f3c3090af6...,[],eotdl
4,Feature,1.0.0,[],2025-05-22 15:55:38.991824,eurocrops-harmonized-only.parquet,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': 'b796162c39391107f3bea7...,[],eotdl
...,...,...,...,...,...,...,...,...,...,...
64,Feature,1.0.0,[],2025-05-22 15:56:37.613855,geoparquet-projected/NL_2020_EC21.parquet,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': '6a05449d02530b5c93d57f...,[],eotdl
65,Feature,1.0.0,[],2025-05-22 15:56:37.797182,geoparquet-projected/AT_2021_EC21.parquet,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': 'cbc9a284e224db073d62a9...,[],eotdl
66,Feature,1.0.0,[],2025-05-22 15:56:39.387310,geoparquet-projected/SK_2021_EC21.parquet,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': '72b7f7495d1213a86b427d...,[],eotdl
67,Feature,1.0.0,[],2025-05-22 15:56:39.493207,geoparquet-projected/LT_2021_EC.parquet,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': '41440405bc0d0c55d33479...,[],eotdl


In [36]:
size = 0
for index, row in gdf.iterrows():
	for k, v in row.assets.items():
		# if not v['href'].startswith('https://'):
		# 	v['href'] = 'https://api.eotdl.com/datasets/682f2d186a29eac175867330/stage/' + v['href']
		# v['href'] = v['href'].replace('/fastdata/kk/dataset/', '')
		print(v['href'])
		# size += v['size']

size

https://api.eotdl.com/datasets/682f2d186a29eac175867330/stage/README.md
https://api.eotdl.com/datasets/682f2d186a29eac175867330/stage/eurocrops-all.pmtiles
https://api.eotdl.com/datasets/682f2d186a29eac175867330/stage/eurocrops-harmonized-only.fgb
https://api.eotdl.com/datasets/682f2d186a29eac175867330/stage/eurocrops-full.duckdb
https://api.eotdl.com/datasets/682f2d186a29eac175867330/stage/eurocrops-harmonized-only.parquet
https://api.eotdl.com/datasets/682f2d186a29eac175867330/stage/unprojected/geoparquet/SI_2021_EC21.parquet
https://api.eotdl.com/datasets/682f2d186a29eac175867330/stage/unprojected/geoparquet/HR_2020_EC21.parquet
https://api.eotdl.com/datasets/682f2d186a29eac175867330/stage/unprojected/geoparquet/DE_NRW_2021_EC21.parquet
https://api.eotdl.com/datasets/682f2d186a29eac175867330/stage/unprojected/geoparquet/SE_2021_EC21.parquet
https://api.eotdl.com/datasets/682f2d186a29eac175867330/stage/unprojected/geoparquet/EE_2021_EC21.parquet
https://api.eotdl.com/datasets/682f2d1

0

In [32]:
gdf.to_parquet(f"/fastdata/kk/dataset/catalog.v1.parquet")

In [33]:
minio_client.fput_object(os.environ['BUCKET'], f'682f2d186a29eac175867330/catalog.v1.parquet', f"/fastdata/kk/dataset/catalog.v1.parquet")

In [10]:
from tqdm import tqdm 

for ix, row in tqdm(gdf.iterrows(), total=len(gdf), desc="Uploading files"):
    tqdm.write(row.id)
    minio_client.fput_object(os.environ['BUCKET'], f'682f2d186a29eac175867330/{row.id}', f"/fastdata/kk/dataset/{row.id}")

Uploading files:   0%|          | 0/69 [00:00<?, ?it/s]

README.md
eurocrops-all.pmtiles


Uploading files:   3%|▎         | 2/69 [00:37<20:57, 18.77s/it]

eurocrops-harmonized-only.fgb


Uploading files:   4%|▍         | 3/69 [04:31<1:59:04, 108.26s/it]

...

Uploading files:  97%|█████████▋| 67/69 [23:15<00:34, 17.34s/it]

geoparquet-projected/LT_2021_EC.parquet


Uploading files:  99%|█████████▊| 68/69 [23:20<00:13, 13.76s/it]

geoparquet-projected/LV_2021_EC21.parquet


Uploading files: 100%|██████████| 69/69 [23:24<00:00, 20.36s/it]


In [33]:
# list all objects in the bucket

collections = list(minio_client.list_objects(os.getenv('BUCKET')))

count = 1
for collection in collections:
	print(collection.object_name, f"({count}/{len(collections)})")
	version = 1
	while True:
		# look for catalog.v{version}.parquet in bucket/collection
		object_name = f"{collection.object_name}catalog.v{version}.parquet"
		try:
			minio_client.stat_object(os.getenv('BUCKET'), object_name)
			print(object_name)
			# download the catalog
			minio_client.fget_object(os.getenv('BUCKET'), object_name, f"kk/{object_name}")
			# backup the catalog
			# minio_client.fput_object(os.getenv('BUCKET'), f"{object_name}.backup", f"kk/{object_name}")
			# read the catalog
			df = gpd.read_parquet(f"kk/{object_name}")
			# update the metadata
			for index, row in df.iterrows():
				for k, v in row.assets.items():
					v['href'] = v['href'].replace('//dev.api.eotdl.com', '//api.eotdl.com')
					# print(k, v)
				break
			# save the catalog
			df.to_parquet(f"kk/{object_name}")
			# upload the catalog
			minio_client.fput_object(os.getenv('BUCKET'), object_name, f"kk/{object_name}")
			version += 1
		except:
			# print(f"No {object_name} found")
			break
	count += 1
	# if count > 2:
	# 	break
		



test.txt (1/94)
645a26564d1c8b7b364ee631/ (2/94)
645a26564d1c8b7b364ee631/catalog.v1.parquet
645d1e4ec3060c653291ce87/ (3/94)
645d1e4ec3060c653291ce87/catalog.v1.parquet
645dff70c2f02fdd88c47b67/ (4/94)
645dff70c2f02fdd88c47b67/catalog.v1.parquet
645e0fb2c2f02fdd88c47b70/ (5/94)
645e0fb2c2f02fdd88c47b70/catalog.v1.parquet
645e3b2dc2f02fdd88c47b77/ (6/94)
645e3b2dc2f02fdd88c47b77/catalog.v1.parquet
645e3e4fc2f02fdd88c47b7a/ (7/94)
645e3e4fc2f02fdd88c47b7a/catalog.v1.parquet
64633fdfc2f02fdd88c47b85/ (8/94)
64633fdfc2f02fdd88c47b85/catalog.v1.parquet
6463589e59028dfdbee33336/ (9/94)
6463589e59028dfdbee33336/catalog.v1.parquet
6463716b59028dfdbee33338/ (10/94)
6463716b59028dfdbee33338/catalog.v1.parquet
646477ac59028dfdbee33341/ (11/94)
646477ac59028dfdbee33341/catalog.v1.parquet
6464899459028dfdbee33344/ (12/94)
6464899459028dfdbee33344/catalog.v1.parquet
64649f3f59028dfdbee3334a/ (13/94)
64649f3f59028dfdbee3334a/catalog.v1.parquet
646640b359028dfdbee3335c/ (14/94)
646640b359028dfdbee333

In [3]:
minio_client.fget_object(os.getenv('BUCKET'), "68074b43c8575682bb134c3e/catalog.v2.parquet", f"kk/backup.parquet")

In [4]:
df = gpd.read_parquet(f"kk/backup.parquet")
# df = gpd.read_parquet(f"~/Desktop/kk/HYPERVIEW2/catalog.v2.parquet")
df

,type,stac_version,stac_extensions,datetime,id,bbox,geometry,assets,links,repository
0,Feature,1.0.0,[],2025-05-22 09:22:11.643191,wavelengths.json,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': '29853bdb7d1c6533bd49fc...,[],eotdl
1,Feature,1.0.0,[],2025-05-22 09:22:11.644079,train_gt.csv,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': '6908af454be9118afdb7de...,[],eotdl
2,Feature,1.0.0,[],2025-05-22 09:22:11.644707,submission.csv,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': '5d9d8c1eee93a0c3be7264...,[],eotdl
3,Feature,1.0.0,[],2025-05-22 09:22:11.645436,README.md-71197,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': 'aa70af75ad12323ee6ef8f...,[],eotdl
4,Feature,1.0.0,[],2025-05-22 09:22:11.645487,starter_pack.py,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': '6800e14da5656f5c4e91b8...,[],eotdl
...,...,...,...,...,...,...,...,...,...,...
9404,Feature,1.0.0,[],2025-05-22 09:22:12.277250,train/msi_satellite/0068.npz,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': 'e0f51b6be90ef1163ffa31...,[],eotdl
9405,Feature,1.0.0,[],2025-05-22 09:22:12.277300,train/msi_satellite/1376.npz,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': 'c06d13dac2f14d0739ae60...,[],eotdl
9406,Feature,1.0.0,[],2025-05-22 09:22:12.277347,train/msi_satellite/1410.npz,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': '834c01160f43072b21629c...,[],eotdl
9407,Feature,1.0.0,[],2025-05-22 09:22:12.277400,train/msi_satellite/1404.npz,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': '3d6f9cc642b20aa939a34b...,[],eotdl


In [5]:
minio_client.stat_object(os.getenv('BUCKET'), f"68074b43c8575682bb134c3e/test/hsi_satellite/0967.npz")

S3Error: S3 operation failed; code: NoSuchKey, message: Object does not exist, resource: /eotdl-data/68074b43c8575682bb134c3e/test/hsi_satellite/0967.npz, request_id: 00000196F7752BC6C1C66984C8055F5C, host_id: 32AAAQAAEAABAAAQAAEAABAAAQAAEAABCSAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA, bucket_name: eotdl-data, object_name: 68074b43c8575682bb134c3e/test/hsi_satellite/0967.npz

In [14]:
for m in tqdm(missing):
    path = f'../kk/prepared_dataset_for_users/{m}'
    minio_client.fput_object(os.getenv('BUCKET'), f"68074b43c8575682bb134c3e/{m}", path)

100%|██████████| 8351/8351 [08:53<00:00, 15.66it/s]


In [15]:
from tqdm import tqdm

missing = []
for index, row in tqdm(df.iterrows(), total=len(df)):
	# print(row.id)
	try:
		minio_client.stat_object(os.getenv('BUCKET'), f"68074b43c8575682bb134c3e/{row.id}")
	except:
		missing.append(row.id)
		print(row.id, len(missing))
	# for k, v in row.assets.items():
	# 	# v['href'] = v['href'].replace('http://api.eotdl.com', 'https://api.eotdl.com')
	# 	# if not v['href'].startswith('https://api.eotdl.com/datasets'): print(k, v)
	# 	print(k, v)
# save the catalog
# df.to_parquet(f"kk/catalog.v2.parquet")

100%|██████████| 9409/9409 [08:02<00:00, 19.50it/s]


In [16]:
len(missing)

0

In [18]:
stats = minio_client.stat_object(os.getenv('BUCKET'), '68074b43c8575682bb134c3e/test/hsi_satellite/0515.npz')

In [19]:
stats.object_name

'68074b43c8575682bb134c3e/test/hsi_satellite/0515.npz'

In [7]:
df = gpd.read_parquet(f"kk/catalog.v2.parquet")
df

,type,stac_version,stac_extensions,datetime,id,bbox,geometry,assets,links,repository
0,Feature,1.0.0,[],2025-04-02 10:34:58.331558,starter_pack.ipynb,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': 'a9d1a60e15beced36f24e7...,[],eotdl
1,Feature,1.0.0,[],2025-04-02 10:34:58.332442,README.md,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': 'aa05385abe52ccd2eeffd9...,[],eotdl
2,Feature,1.0.0,[],2025-04-02 10:34:58.332511,starter_pack.pdf,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': '28c813f9a0b62213548222...,[],eotdl
3,Feature,1.0.0,[],2025-04-02 10:34:58.332669,catalog.v1.parquet,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': 'c3038a3d6fa00ad3133779...,[],eotdl
4,Feature,1.0.0,[],2025-04-02 10:34:58.332868,test_data/77.npz,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': '743561ae45eab04e8418e3...,[],eotdl
...,...,...,...,...,...,...,...,...,...,...
2887,Feature,1.0.0,[],2025-04-02 10:35:01.933934,train_data/train_data/1410.npz,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': 'ea5cd6770fe8a287255cac...,[],eotdl
2888,Feature,1.0.0,[],2025-04-02 10:35:01.936311,train_data/train_data/184.npz,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': '02557206016e4fe875da88...,[],eotdl
2889,Feature,1.0.0,[],2025-04-02 10:35:01.936512,train_data/train_data/190.npz,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': '7f3bd6e3be91a19c8cfa74...,[],eotdl
2890,Feature,1.0.0,[],2025-04-02 10:35:01.936718,train_data/train_data/1404.npz,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': '247252e3d357cbed7a6453...,[],eotdl


In [8]:
df.assets.sample(1).iloc[0]

{'asset': {'checksum': 'db1a64b0b52e3693a09509e46cf6ed445ee031cc',
  'href': 'https://api.eotdl.com/datasets/67d182a43926af70561e6d36/stage/test_data/819.npz',
  'size': 3220482,
  'timestamp': datetime.datetime(2025, 4, 2, 10, 34, 58, 522841)}}

In [9]:
minio_client.fput_object(os.getenv('BUCKET'), "67d182a43926af70561e6d36/catalog.v2.parquet", f"kk/catalog.v2.parquet")

In [45]:
minio_client.fget_object(os.getenv('BUCKET'), "65f407248e68180349152973/catalog.v2.parquet", f"kk/backup.parquet")

In [46]:
df = gpd.read_parquet(f"kk/backup.parquet")
df

,type,stac_version,stac_extensions,datetime,id,bbox,geometry,assets,links,repository
0,Feature,1.0.0,[],2025-03-25 13:49:59.589565,model.onnx,"{'xmax': 0.0, 'xmin': 0.0, 'ymax': 0.0, 'ymin'...",POLYGON EMPTY,{'asset': {'checksum': '99581a42cd56f66b1649f5...,[],eotdl


In [47]:
df.assets.sample(1).iloc[0]

{'asset': {'checksum': '99581a42cd56f66b1649f5246663cea7633a1007',
  'href': 'https://api.eotdl.com/models/65f407248e68180349152973/stage/model.onnx',
  'size': 130011868,
  'timestamp': '2024-05-30T17:14:39.987000'}}